## CS:GO Form Prediction

In [204]:
import pandas as pd
import numpy as np
import csv
import theano as th

from theano import tensor as T

In [2]:
dat = pd.read_csv('matchdata.csv',encoding = "ISO-8859-1")

C:\Users\Maximilian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (82,83,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Variable Names...

In [3]:
list(dat.columns.values)

['Day',
 'Match ID',
 'Map',
 'Team 1',
 'Team 2',
 'Winning Team',
 'Score Team 1 as T',
 'Score Team 2 as CT',
 'Score Team 1 as CT',
 'Score Team 2 as T',
 'Player 1 Name',
 'Player 1 UserName',
 'Player 1 Kills',
 'Player 1 Deaths',
 'Player 1 KDA',
 'Player 1 ADR',
 'Player 2 Name',
 'Player 2 UserName',
 'Player 2 Kills',
 'Player 2 Deaths',
 'Player 2 KDA',
 'Player 2 ADR',
 'Player 3 Name',
 'Player 3 UserName',
 'Player 3 Kills',
 'Player 3 Deaths',
 'Player 3 KDA',
 'Player 3 ADR',
 'Player 4 Name',
 'Player 4 UserName',
 'Player 4 Kills',
 'Player 4 Deaths',
 'Player 4 KDA',
 'Player 4 ADR',
 'Player 5 Name',
 'Player 5 UserName',
 'Player 5 Kills',
 'Player 5 Deaths',
 'Player 5 KDA',
 'Player 5 ADR',
 'Player 6 Name',
 'Player 6 UserName',
 'Player 6 Kills',
 'Player 6 Deaths',
 'Player 6 KDA',
 'Player 6 ADR',
 'Player 7 Name',
 'Player 7 UserName',
 'Player 7 Kills',
 'Player 7 Deaths',
 'Player 7 KDA',
 'Player 7 ADR',
 'Player 8 Name',
 'Player 8 UserName',
 'Player 8 

In [93]:
# Need a function that pulls out only a certain teams data:

def team_data(team): # probably by index...
    df = dat.loc[(dat.ix[:,3] == team) |  (dat.ix[:,4] == team) ]
    return(df)

# make a player data one...

def player_data(player): # can define by indx later as well...
    df = dat.loc[(dat.ix[:,11] == player) |  (dat.ix[:,17] == player)|  (dat.ix[:,23] == player)|  (dat.ix[:,29] == player)|  (dat.ix[:,35] == player)|  (dat.ix[:,41] == player) |  (dat.ix[:,47] == player) |  (dat.ix[:,53] == player) |  (dat.ix[:,59] == player) |  (dat.ix[:,65] == player)]
    return(df)

# make function that says which team a player is on (1 or 2)...
def player_side(player, match): # match is the match # as integer

    dato = player_data(player).reset_index(drop=True) 
    N = len(dato)
    i = match
    
    side = []
    # loop over each row...
    if( (dato.ix[i,11] == player) |  (dato.ix[i,17] == player)|  (dato.ix[i,23] == player)|  (dato.ix[i,29] == player)|  (dato.ix[i,35] == player) ):
        side = 1
    elif( (dato.ix[i,41] == player) |  (dato.ix[i,47] == player) |  (dato.ix[i,53] == player) |  (dato.ix[i,59] == player) |  (dato.ix[i,65] == player) ):
        side = 2
    else:
        side = 0
    return(side)


def team_score(player, match):
    '''
    Returns team scores, enemy scores and team deaths (TD)...
    '''
    
    side = player_side(player, match)
    dato = player_data(player).reset_index(drop=True) 
    i = match
    if(side == 1):
        teamscore = dato.ix[match,6] +dato.ix[match,8]
        enemyscore = dato.ix[match,7] + dato.ix[match,9]
        TK = (dato.ix[i,12]) + (dato.ix[i,18])+  (dato.ix[i,24])+(dato.ix[i,30])+ (dato.ix[i,36])
        TD = (dato.ix[i,13] == player) + (dato.ix[i,19])+  (dato.ix[i,25])+(dato.ix[i,31])+ (dato.ix[i,37])
        
    elif(side ==2 ):
        teamscore = dato.ix[match,7] +dato.ix[match,9]
        enemyscore = dato.ix[match,6] +dato.ix[match,8]
        TK = dato.ix[i,42] + (dato.ix[i,48])+  (dato.ix[i,54])+(dato.ix[i,60] )+ (dato.ix[i,66])
        TD = dato.ix[i,43]+ (dato.ix[i,19])+  (dato.ix[i,55])+(dato.ix[i,61] )+ (dato.ix[i,67])
        
    else:
        teamscore = "ERROR"
        enemyscore = "UNLUGGY UCE"
        
        
    return( teamscore, enemyscore, TK, TD)



In [96]:
TS, ES, tk, td = team_score('GeT_RiGhT', 3)

print('team score is:', TS )
print('enemy score is:', ES )
print('team kills are:', tk )
print('team deaths are:', td)


team score is: 16
enemy score is: 11
team kills are: 104.0
team deaths are: 86.0


In [79]:

np.sum(dat.ix[0,([12,18,24,30])])

70

In [137]:
'''
get_metrics returns the players kills, KDR, teamscore, enemyscore, team kills and enemy kills at specified time...
'''
def get_metrics(player, match):
    TS, ES, tk, td = team_score(player, match)
    # get the new player 
    dato = player_data(player).reset_index(drop=True) 
    i = match
    if( dato.ix[i,11] == player ):
        kills = dato.ix[i,12]
        deaths = dato.ix[i,13]
        kd = dato.ix[i,12]/dato.ix[i,13]    
    elif( dato.ix[i,17] == player ):
        kills = dato.ix[i,18]
        deaths = dato.ix[i,19]
        kd = dato.ix[i,18]/dato.ix[i,19]
    elif( dato.ix[i,23] == player ):
        kills = dato.ix[i,24]    
        deaths = dato.ix[i,25]
        kd = dato.ix[i,24]/dato.ix[i,25]
    elif( dato.ix[i,29] == player ):
        kills = dato.ix[i,30]  
        deaths = dato.ix[i,31]
        kd = dato.ix[i,30]/dato.ix[i,31]
    elif( dato.ix[i,35] == player ):
        kills = dato.ix[i,36]
        deaths = dato.ix[i,37]
        kd = dato.ix[i,36]/dato.ix[i,37]
    elif( dato.ix[i,41] == player ):
        kills = dato.ix[i,42]    
        deaths = dato.ix[i,43]
        kd = dato.ix[i,42]/dato.ix[i,43]
    elif( dato.ix[i,47] == player ):
        kills = dato.ix[i,48]       
        deaths = dato.ix[i,49]
        kd = dato.ix[i,48]/dato.ix[i,49]
    elif( dato.ix[i,53] == player ):
        kills = dato.ix[i,54]      
        deaths = dato.ix[i,55]
        kd = dato.ix[i,54]/dato.ix[i,55]
    elif( dato.ix[i,59] == player ):
        kills = dato.ix[i,60] 
        deaths = dato.ix[i,61]
        kd = dato.ix[i,60]/dato.ix[i,61]
    elif( dato.ix[i,65] == player ):
        kills = dato.ix[i,66] 
        deaths = dato.ix[i,67]
        kd = dato.ix[i,66]/dato.ix[i,67]        
     
    k_TK = kills / tk
    kDR_A = kd / (np.log(1+TS)*(TS - ES+1))
    
    return(kills, deaths, kd, TS, ES, tk ,td, k_TK, kDR_A)            
                
'''
Now make function that makes CSV/df for the player and their past performance...
'''   

def make_inputs(player): # uses the player and gets all inputs from this...
    datta = player_data(player).reset_index(drop=True) 
    N = np.shape(datta)[0] # length of the data frame...
    
    # initialize the vectors...
    kill = []
    kdr = []
    scorediff = []
    kill_cont = [] # contribution of kills
    death_cont = [] # contriubution of deaths
    kdr_adjust = []
    
    print(N)
    
    # loop over each row to get data:
    for row in range(N):
        k,d,KDR, TS,ES,tk,td,k_TK,kDR_A = get_metrics('GeT_RiGhT', row)
        kill.append(k)
        kdr.append(KDR)
        scorediff.append(TS-ES)
        kill_cont.append(k_TK)
        death_cont.append(d/td)
        kdr_adjust.append(kDR_A)
        
        
    # add all as columns to the dataframe...    
    newdat = pd.DataFrame([kill,kdr,scorediff,kill_cont, death_cont, kdr_adjust])
    
    # but data is a tensor - so turn into tensor
    
    return(newdat)

In [186]:
def get_tensor(dataframe, n ):
        # dataframe is the one resulting from newdat in "make_inputs"
        df = np.array(dataframe)
        #KK = np.reshape(df,(1,np.shape(df)[1],np.shape(df)[0]) )
        KK
        # Now make each sample and add it to the final dimension of the tensor...
        # window 
        KK[1] =  KK[0]# this is the sample one...
        
        
        return(np.shape(KK))

In [208]:
N = 100
T = 500
K = 6
# iniitalize tensor with 
tens = np.zeros((N,T,K))

tens[0] = np.random.randn(T,K) # this is the dataframe...
tens[1] = np.random.randn(T,K) 

tens


array([[[  5.54201874e-01,  -7.70221264e-01,  -5.88703300e-01,
           5.48301581e-01,  -1.60404174e-02,   5.07222867e-01],
        [  5.71043996e-01,   1.06462487e+00,   2.51099279e+00,
           1.63786461e+00,  -2.43862786e-01,   2.00706816e-02],
        [ -1.58443993e+00,  -4.65270889e-01,  -1.49810223e+00,
          -1.25479880e+00,   5.39618642e-01,   1.22676464e+00],
        ..., 
        [ -2.16715270e-01,  -7.81433811e-01,  -1.10648090e+00,
           2.57424519e-01,  -7.48413990e-01,  -1.83889374e+00],
        [ -1.30175322e-01,  -1.55635211e+00,  -2.93113746e-01,
           1.76678709e-01,  -4.40419890e-01,  -7.94245454e-01],
        [ -1.18540727e+00,  -2.35951597e-03,   1.74477131e+00,
           1.62563023e+00,   1.88703811e+00,  -8.84301127e-01]],

       [[ -1.22543547e-01,   7.34860297e-02,  -5.91010177e-01,
           1.57823028e+00,   4.89367570e-01,  -8.88451508e-01],
        [  4.35838608e-01,   1.53749526e+00,   1.17793360e+00,
           1.47912224e+00,   1.1

In [203]:
#n = 1 # number of samples...
#df = np.array(df)
#KK = np.reshape(df,(n,np.shape(df)[1],np.shape(df)[0]) )
#np.shape(KK)
J = [[[[1,2],[1,2]]],[[[1,2]]]]
J[0]
print(np.shape(J))
#get_tensor(df,2)
print(J[0])
#print(KK[0][0])

(2, 1)
[[[1, 2], [1, 2]]]


In [138]:
#np.shape((player_data('GeT_RiGhT').reset_index(drop=True) ))[0]
df = make_inputs('GeT_RiGhT')
df.to_csv('getright data.csv') # save this to csv

537


In [139]:
def get_targ(data_piece):
    '''
    This code splits the data series into a target and remaining inputs 
    '''
    




In [74]:
# win ratio metric:

def win_ratio(team):
    # get team data...
    dato = team_data(team).reset_index(drop=True)
    #print('The subset data length is:', len(dato))
    # then iterate over every row:
    WL = []
    #print(np.shape(data)[0])
    for row in range(len(dato)):
        # print(dat.ix[row,5])    # this here jsut for testing...
        if (dato.ix[row,5]) == team : # this is if they are the winning team - name is in the winning team col...
            WL.append(1)
            
        else:
            WL.append(0)
            
    #return(WL)
    return(np.sum(WL)/len(WL), np.std(WL)) # this is the Win ratio metric... and variance (STD)
    

In [77]:
win_ratio('EnVyUs') # returns mean and stdev

(0.60546875, 0.48874977521574114)

In [96]:
# compile a lookup table of each team - unique team ID and team name...

def table():
    '''
    This defines the total unique teams and their idx numbers...
    '''
    # total teams
    teams = pd.concat([dat.ix[:,3],dat.ix[:,4]])
    teams= teams.reset_index(drop=True)
    teams = pd.DataFrame(pd.unique(teams))
    print(teams)

In [97]:
table()

                      0
0                sylloM
1          Epsilon Hype
2             Fragsport
3       ALTERNATE aTTaX
4                  LDLC
5              NiceShot
6                  ENCE
7               Epsilon
8                fnatic
9                    SK
10          Preparation
11                   G2
12         dream[S]cape
13        Natus Vincere
14              B.O.O.T
15             dignitas
16          AVANT GARDE
17                  NiP
18            Athletico
19                 New4
20          HellRaisers
21          VG.CyberZen
22              onestop
23               Exile5
24           Virtus.pro
25                TyLoo
26            iGame.com
27             FlipSid3
28                 k1ck
29            Immortals
...                 ...
1331     kRoGhs KanonEr
1332           MIXERINO
1333            madjicK
1334   Eternal Conflict
1335       The Flying V
1336          Deponeret
1337            Vikings
1338            DEMGUNZ
1339   kassad's Legends
1340            